In [5]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import os
import time
import unidecode

In [2]:
def get_roster_url(team):
    team_new = team.lower().replace(' ', '-')
    roster_url = 'https://www.fangraphs.com/roster-resource/depth-charts/' + team_new
    return roster_url

# Scrape active roster
def get_roster_source(roster_url,driver):
    driver.set_page_load_timeout(5)
    try :
        driver.get(roster_url)
    except TimeoutException as e:
        # try again
        driver.get(roster_url)
    time.sleep(5)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    table = soup.find("div", {"class":"depth-chart-wrapper is-spring-training"})
    #driver.quit()
    return table

def get_player_url(roster_table):
    player = []
    player_url = []
    
    dcrt = roster_table.find_all('div',{"class":"depth-charts__roster-tables"})
    
    # first seven tables are the 40 man roster
    for i in range(7):        
        for row2 in dcrt[i].tbody.find_all('td',{"data-stat":"PLAYER"}):
                    
            #player list
            player_tmp = row2.text
            player.append(player_tmp)
                    
            #player url list
            player_url_tmp = row2.find('a')['href']
            size = len(player_url_tmp)
            player_url_tmp = 'https:' + player_url_tmp[:size - 5]
            player_url.append(player_url_tmp)
                    
    return player, player_url

# Static function for stat_cat == 'advanced' and 'batted-ball'

def batting(url, season, driver):
    #driver = webdriver.Firefox(executable_path = geckodriver)     
    batting_url = url + 'splits?season=' + str(season)
    
    driver.set_page_load_timeout(5)
    try :
        driver.get(batting_url)
    except TimeoutException as e:
        try :
            driver.get(batting_url)
        except TimeoutException as e:
            # try again
            driver.get(batting_url)
     
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #In case of rediction to pitch 
    if 'position=P' in driver.current_url:
        real_batting_url = soup.find("div", {"id":"portal-player-pages-menu"}).find("ul",
                        {"class":"menu-player-page__batpitch"}).find_all("a")[0]['href']
        real_batting_url = 'https://www.fangraphs.com/' + real_batting_url + '&season=' + str(season)
        driver.get(real_batting_url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
    table_advanced = soup.find("div", {"id":'advanced',"class":"player-page-table"})
    table_battedball = soup.find("div", {"id":'batted-ball',"class":"player-page-table"})
    
    if (table_advanced is None) or (table_battedball is None):
        print(url,'table_advanced is None')
        # If encountering error page, use Derek Lowe as a dummy pitcher 
        dummy_batting_url = 'https://www.fangraphs.com/players/derek-lowe/199/splits?position=PB&season=' + str(season) 
        driver.get(dummy_batting_url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')    
        table_advanced = soup.find("div", {"id":'advanced',"class":"player-page-table"})
        table_battedball = soup.find("div", {"id":'batted-ball',"class":"player-page-table"})
        
    #driver.quit()
    return table_advanced, table_battedball

# Static function for stat_cat == 'advanced' and 'batted-ball'
def pitching(url, season, driver):

    #driver = webdriver.Firefox(executable_path = geckodriver)    
    
    #Set position=P
    pitching_url = url + 'splits?position=P&season=' + str(season)
    
    driver.set_page_load_timeout(4)
    try :
        driver.get(pitching_url)
    except TimeoutException as e:
        try :
            driver.get(pitching_url)
        except TimeoutException as e:
            # try again
            driver.get(pitching_url)

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    table_advanced = soup.find("div", {"id":'advanced',"class":"player-page-table"})
    table_battedball = soup.find("div", {"id":'batted-ball',"class":"player-page-table"})
    if (table_advanced is None) or (table_battedball is None):
        print(pitching_url,'table is None')
    # If encountering error page, use Derek Lowe as a dummy pitcher 
        dummy_url = 'https://www.fangraphs.com/players/derek-lowe/199/splits?position=PB&season=' + str(season) 
        driver.get(dummy_url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')    
        table_advanced = soup.find("div", {"id":'advanced',"class":"player-page-table"})
        table_battedball = soup.find("div", {"id":'batted-ball',"class":"player-page-table"})
    
    #driver.quit()
    return table_advanced, table_battedball

# append new info to df

def advanced_batter_table(table, stat_cat, stat, df, team, player, bat_pitch):
    
    for row in table.tbody.find_all('tr'): 
    # Find all data for each column
        if (row.find_all('th') != []):
    # Replace "Handedness" with function input
            stat_row = row.find_all('th')[1].text
            #print(stat_row)
        if all(stat_row != x for x in stat):
            #if all(row.find_all('th')[1].text != x for x in stat):
            #if (row.find_all('th')[1].text != stat):
            continue
        columns = row.find_all('td')
        if (columns == []):
            continue
        #bat_pitch = Bat_Pitch
        season = columns[0].text.strip()
        split = columns[1].text.strip()
        bb = columns[2].text.strip()
        k_per = columns[3].text.strip()
        bb_k = columns[4].text.strip()
        avg = columns[5].text.strip()
        obp = columns[6].text.strip()
        slg = columns[7].text.strip()
        ops = columns[8].text.strip()
        iso = columns[9].text.strip()
        babip = columns[10].text.strip()
        wrc = columns[11].text.strip()
        wraa = columns[12].text.strip()
        woba = columns[13].text.strip()
        wrc_plus = columns[14].text.strip()
        df = df.append({'Team': team, 'Player': player, 'Bat/Pitch':bat_pitch, 'Stat_cat': stat_cat,'Stat': stat_row, 'Season':season, 'Split':split, 'BB%':bb, 'K%':k_per, 'BB/K':bb_k ,'AVG':avg, 
                    'OBP':obp, 'SLG':slg, 'OPS':ops, 'ISO':iso, 'BABIP':babip, 'wRC':wrc, 
                    'wRAA':wraa, 'wOBA':woba, 'wRC+':wrc_plus}, ignore_index=True)
    return df

In [13]:
# Static function for stat_cat == 'advanced' and 'batted-ball'

def fg_scrape(team, season, bat_pitch, stat = ['Handedness', 'Home / Away', 'Monthly']):
    start_time = time.time()
    geckodriver = "C:\Program Files (x86)\geckodriver"
    os.environ["webdriver.gecko.driver"] = geckodriver
    driver = webdriver.Firefox(executable_path = geckodriver)
    
    roster_url = get_roster_url(team)
    
    #timeout1 = time.time() + 5
    #while time.time() < timeout1:
    roster_table = get_roster_source(roster_url, driver)
    
    # Full team name
    team_tmp = roster_table.find('h2',{"class":"rr-header undefined"}).text
    team_tmp = team_tmp[:len(team_tmp) - 6]
    
    player, player_url = get_player_url(roster_table)
    
    # URL time
    print("--- getting url takes %s seconds ---" % (time.time() - start_time))

    if (bat_pitch == 'bat'):    
        
        # Generate empty df
        df_advanced = pd.DataFrame(columns=['Team', 'Player','Stat_cat', 'Bat/Pitch','Stat', 'Season', 
                           'Split', 'BB%', 'K%', 'BB/K' ,'AVG', 'OBP', 
                           'SLG', 'OPS', 'ISO', 'BABIP', 'wRC', 'wRAA', 'wOBA', 'wRC+'])
        df_battedball = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'GB/FB','LD%','GB%','FB%','IFFB%','HR/FB','IFH%','BUH%',
                           'Pull%','Cent%','Oppo%','Soft%','Med%','Hard%','Pitches','Balls','Strikes'])         
        # Loop over players
        for i in range(len(player_url)):
                #table = batting(player_url[i], season, stat_cat, driver)
            print(player[i])
            
            table_advanced, table_battedball = batting(player_url[i], season, driver)
            
            print("--- scraping tables takes %s seconds ---" % (time.time() - start_time))
            
            df_advanced = advanced_batter_table(table_advanced, 'advanced', stat, df_advanced, team = team_tmp, player = player[i], bat_pitch = 'Batter')
            print("--- formatting first table takes %s seconds ---" % (time.time() - start_time))
            
            df_battedball = battedball_table(table_battedball, 'batted-ball', stat, df_battedball, team = team_tmp, player = player[i], bat_pitch = 'Batter')   
            print("--- formatting second table takes %s seconds ---" % (time.time() - start_time))
        
        df_advanced.to_csv("fangraphsExport\\ScrapeGround\\" + team + '_' + str(season) + "_bat_advanced.csv", index = False)
        df_battedball.to_csv("fangraphsExport\\ScrapeGround\\" + team + '_' + str(season) + "_bat_battedball.csv", index = False)
        
        #return df_advanced, df_battedball
   

    if (bat_pitch == 'pitch'):
        df_advanced = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'K/9', 'BB/9', 'K/BB','HR/9','K%','BB%','K-BB%','AVG',
                           'WHIP','BABIP','LOB%','FIP','xFIP'])
        df_battedball = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'GB/FB','LD%','GB%','FB%','IFFB%','HR/FB','IFH%','BUH%',
                           'Pull%','Cent%','Oppo%','Soft%','Med%','Hard%','Pitches','Balls','Strikes'])        
        
        for i in range(len(player_url)):
            print(player[i])

            table_advanced, table_battedball = pitching(player_url[i], season, driver)
            df_advanced = advanced_pitcher_table(table_advanced, 'advanced', stat, df_advanced, team = team_tmp, player = player[i], bat_pitch = 'Pitcher')
            df_battedball = battedball_table(table_battedball, 'batted-ball', stat, df_battedball, team = team_tmp, player = player[i], bat_pitch = 'Pitcher')
            print("--- formatting second table takes %s seconds ---" % (time.time() - start_time))
        df_advanced.to_csv("fangraphsExport\\ScrapeGround\\" + team + '_' + str(season) + "_pitch_advanced.csv" ,index = False)
        df_battedball.to_csv("fangraphsExport\\ScrapeGround\\" + team + '_' + str(season) + "_pitch_battedball.csv",index = False)
        
    driver.quit()
    print("--- %s seconds ---" % (time.time() - start_time))
        #return df_advanced, df_battedball

# append new info to df

def advanced_pitcher_table(table, stat_cat, stat, df, team, player, bat_pitch):
    
    for row in table.tbody.find_all('tr'): 
    # Find all data for each column
        if (row.find_all('th') != []):
    # Replace "Handedness" with function input
            stat_row = row.find_all('th')[1].text
            #print(stat_row)
        if all(stat_row != x for x in stat):
            #if all(row.find_all('th')[1].text != x for x in stat):
            #if (row.find_all('th')[1].text != stat):
            continue
        columns = row.find_all('td')
        if (columns == []):
            continue
        #bat_pitch = Bat_Pitch
        season = columns[0].text.strip()
        split = columns[1].text.strip()  
        k_9 = columns[2].text.strip()
        bb_9 = columns[3].text.strip()
        k_bb = columns[4].text.strip()
        hr_9 = columns[5].text.strip()
        k = columns[6].text.strip()
        bb = columns[7].text.strip()
        kbb = columns[8].text.strip()
        avg = columns[9].text.strip()
        whip = columns[10].text.strip()
        babip = columns[11].text.strip()
        lob = columns[12].text.strip()
        fip = columns[13].text.strip()
        xfip = columns[14].text.strip()
        
        df = df.append({'Team': team, 'Player': player, 'Bat/Pitch':bat_pitch, 'Stat_cat': stat_cat,'Stat': stat_row, 'Season':season, 'Split':split, 
                        'K/9':k_9, 'BB/9':bb_9, 'K/BB':k_bb,'HR/9':hr_9,'K%':k,'BB%':bb,'K-BB%':kbb,'AVG':avg,
                           'WHIP':whip,'BABIP':babip,'LOB%':lob,'FIP':fip,'xFIP':xfip}, ignore_index=True)
    return df

def battedball_table(table, stat_cat, stat, df, team, player, bat_pitch):
    
    for row in table.tbody.find_all('tr'): 
    # Find all data for each column
        if (row.find_all('th') != []):
    # Replace "Handedness" with function input
            stat_row = row.find_all('th')[1].text
            #print(stat_row)
        if all(stat_row != x for x in stat):
            #if all(row.find_all('th')[1].text != x for x in stat):
            #if (row.find_all('th')[1].text != stat):
            continue
        columns = row.find_all('td')
        if (columns == []):
            continue
        #bat_pitch = Bat_Pitch
        season = columns[0].text.strip()
        split = columns[1].text.strip()
        gb_fb = columns[2].text.strip()
        ld = columns[3].text.strip()
        gb = columns[4].text.strip()
        fb = columns[5].text.strip()
        iffb = columns[6].text.strip()
        hr_fb = columns[7].text.strip()
        ifh = columns[8].text.strip()
        buh = columns[9].text.strip()
        pull = columns[11].text.strip()
        cent = columns[12].text.strip()
        oppo = columns[13].text.strip()
        soft = columns[15].text.strip()
        med = columns[16].text.strip()
        hard = columns[17].text.strip()
        pitches = columns[19].text.strip()
        balls = columns[20].text.strip()
        strikes = columns[21].text.strip()
             
        df = df.append({'Team': team, 'Player': player,'Bat/Pitch':bat_pitch, 'Stat_cat': stat_cat,'Stat': stat_row, 'Season':season,
                           'Split':split, 'GB/FB':gb_fb,'LD%':ld, 'GB%':gb,'FB%':fb,'IFFB%':iffb, 'HR/FB':hr_fb,
                        'IFH%':ifh,'BUH%':buh,'Pull%':pull,'Cent%':cent,'Oppo%':oppo,'Soft%':soft,
                        'Med%':med,'Hard%':hard,'Pitches':pitches,'Balls':balls,'Strikes':strikes}, 
                       ignore_index=True)
    return df


# Run scraping function

In [16]:
teams = ['orioles','red sox','white sox', 'guardians', 'tigers', 'astros', 'royals', 'angels', 'twins', 'yankees',
            'athletics', 'mariners' , 'rays', 'rangers', 'blue jays', 'diamondbacks', 'braves','cubs','reds',
             'rockies','dodgers', 'marlins', 'brewers', 'mets', 'phillies', 'pirates', 'padres', 'giants', 'cardinals', 
             'nationals']

seasons = ['2022']
#stat = ['Handedness', 'Home / Away', 'Monthly']
stat = ['Handedness']
for team in teams:
    for season in seasons:
        fg_scrape(team, season, 'bat', stat)
        fg_scrape(team, season, 'pitch', stat)

--- getting url takes 11.865327835083008 seconds ---
Cedric Mullins
--- scraping tables takes 13.764633655548096 seconds ---
--- formatting first table takes 13.777571678161621 seconds ---
--- formatting second table takes 13.793532133102417 seconds ---
Trey Mancini
--- scraping tables takes 15.333129167556763 seconds ---
--- formatting first table takes 15.348087787628174 seconds ---
--- formatting second table takes 15.362050771713257 seconds ---
Anthony Santander
--- scraping tables takes 17.7940936088562 seconds ---
--- formatting first table takes 17.805063724517822 seconds ---
--- formatting second table takes 17.81802773475647 seconds ---
Ryan Mountcastle
--- scraping tables takes 20.236964225769043 seconds ---
--- formatting first table takes 20.245968341827393 seconds ---
--- formatting second table takes 20.256937980651855 seconds ---
Austin Hays
--- scraping tables takes 22.22729229927063 seconds ---
--- formatting first table takes 22.2382390499115 seconds ---
--- formattin

--- formatting second table takes 16.64578127861023 seconds ---
Adley Rutschman
--- formatting second table takes 17.57095170021057 seconds ---
Ramón Urías
--- formatting second table takes 19.061590671539307 seconds ---
Rougned Odor
--- formatting second table takes 19.755962371826172 seconds ---
Jorge Mateo
--- formatting second table takes 20.401302814483643 seconds ---
Robinson Chirinos
--- formatting second table takes 21.12786602973938 seconds ---
Jonathan Araúz
--- formatting second table takes 21.718125104904175 seconds ---
Tyler Nevin
--- formatting second table takes 22.32080364227295 seconds ---
Ryan McKenna
--- formatting second table takes 22.925103425979614 seconds ---
Jordan Lyles
--- formatting second table takes 24.256545305252075 seconds ---
Spenser Watkins
--- formatting second table takes 25.31806468963623 seconds ---
Tyler Wells
--- formatting second table takes 26.64059829711914 seconds ---
Dean Kremer


InvalidSessionIdException: Message: Tried to run command without establishing a connection


## Check missing players of each team, then append to current file

In [18]:
def load_team_mapping():
    os.chdir('T:\Baseball')
    df = pd.read_csv('teams_mapping.csv', index_col=False)
    return df

def get_all_players(teams):
    geckodriver = "C:\Program Files (x86)\geckodriver"
    os.environ["webdriver.gecko.driver"] = geckodriver
    
    driver = webdriver.Firefox(executable_path = geckodriver)

    # Get all players
    player_all = []
    player_url_all = []
    team_all = []
    team_abbr = []
    df_team_abbr = load_team_mapping()
    for team in teams:
        team_abbr_tmp = df_team_abbr[df_team_abbr['Teams'] == team]['Teams_abbr'].to_string()[5:]
        roster_url = get_roster_url(team)
        roster_table = get_roster_source(roster_url, driver)
    
        player, player_url = get_player_url(roster_table)
        for p in player:
            player_all.append(p)
            team_all.append(team)
            team_abbr.append(team_abbr_tmp)
        for u in player_url:
            player_url_all.append(u)
            
    df_player_all = pd.DataFrame({'Player':player_all, 'Player_url': player_url_all, 'Team':team_all, 'Team_abbr': team_abbr})       
    return df_player_all

#Append function

# Loop over pitcher advanced file
def find_missing_pitchers(directory, df_player_all, stat_cat, stat, season):
    
    geckodriver = "C:\Program Files (x86)\geckodriver"
    os.environ["webdriver.gecko.driver"] = geckodriver

    driver = webdriver.Firefox(executable_path = geckodriver)

    df_advanced = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'K/9', 'BB/9', 'K/BB','HR/9','K%','BB%','K-BB%','AVG',
                           'WHIP','BABIP','LOB%','FIP','xFIP'])
    df_battedball = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'GB/FB','LD%','GB%','FB%','IFFB%','HR/FB','IFH%','BUH%',
                           'Pull%','Cent%','Oppo%','Soft%','Med%','Hard%','Pitches','Balls','Strikes'])   
    os.chdir(directory)
    for filename in os.listdir(directory):

        if 'csv' not in filename:
            continue
        f = os.path.join(directory, filename)
        df_read = pd.read_csv(f)
        orig_file_size = len(df_read)
        #team_to_check = df_read['Team'][0]
        players_to_check = df_read['Player'].tolist()
        team_to_check = filename[:filename.index('_2022')]
        
        team_name_formal = df_read['Team'][0]
        all_active_players_of_team = df_player_all[df_player_all['Team'] == team_to_check]['Player'].tolist()
              
        for i in range(len(all_active_players_of_team)):  
            if all_active_players_of_team[i] not in players_to_check: # if current online roster player not in local
                print(team_to_check, all_active_players_of_team[i])
                table_advanced, table_battedball = pitching(df_player_all['Player_url'][i], season, driver)
    
                if stat_cat == 'advanced':
                    df_advanced_player = advanced_pitcher_table(table_advanced, stat_cat, stat, df_advanced, team = team_name_formal, player = all_active_players_of_team[i], bat_pitch = 'Pitcher')
                    df_read = df_read.append(df_advanced_player)
            
                if stat_cat == 'batted-ball':
                    df_battedball_player = battedball_table(table_battedball, stat_cat, stat, df_battedball, team = team_name_formal, player = all_active_players_of_team[i], bat_pitch = 'Pitcher')
                    df_read = df_read.append(df_battedball_player)
                    
        if (stat_cat == 'advanced') and (len(df_read) != orig_file_size):
            print(len(df_read) - orig_file_size)
            df_read.to_csv(team_to_check + '_' + str(season) + "_pitch_advanced.csv" ,index = False)
            print('export done')
        if (stat_cat == 'batted-ball') and (len(df_read) != orig_file_size):    
            print(len(df_read) - orig_file_size)
            df_read.to_csv(team_to_check + '_' + str(season) + "_pitch_battedball.csv",index = False)
            print('export done')
    driver.quit()



def find_missing_batters(directory, df_player_all, stat_cat, stat, season):
    
    geckodriver = "C:\Program Files (x86)\geckodriver"
    os.environ["webdriver.gecko.driver"] = geckodriver

    driver = webdriver.Firefox(executable_path = geckodriver)

    df_advanced = pd.DataFrame(columns=['Team', 'Player','Stat_cat', 'Bat/Pitch','Stat', 'Season', 
                           'Split', 'BB%', 'K%', 'BB/K' ,'AVG', 'OBP', 
                           'SLG', 'OPS', 'ISO', 'BABIP', 'wRC', 'wRAA', 'wOBA', 'wRC+'])
    df_battedball = pd.DataFrame(columns=['Team', 'Player','Stat_cat','Bat/Pitch','Stat', 'Season', 
                           'Split', 'GB/FB','LD%','GB%','FB%','IFFB%','HR/FB','IFH%','BUH%',
                           'Pull%','Cent%','Oppo%','Soft%','Med%','Hard%','Pitches','Balls','Strikes'])  
    os.chdir(directory)
    for filename in os.listdir(directory):
        if 'csv' not in filename:
            continue
        f = os.path.join(directory, filename)
        df_read = pd.read_csv(f)
        orig_file_size = len(df_read)
        #team_to_check = df_read['Team'][0]
        players_to_check = df_read['Player'].tolist()
        team_to_check = filename[:filename.index('_2022')]
        team_name_formal = df_read['Team'][0]
        all_active_players_of_team = df_player_all[df_player_all['Team'] == team_to_check]['Player'].tolist()

        for i in range(len(all_active_players_of_team)):
            if all_active_players_of_team[i] not in players_to_check:
                print(team_to_check, all_active_players_of_team[i])
                table_advanced, table_battedball = batting(df_player_all['Player_url'][i], season, driver)
                #print(table_advanced)

                if stat_cat == 'advanced':
                    df_advanced_player = advanced_batter_table(table_advanced, stat_cat, stat, df_advanced, team = team_name_formal, player = all_active_players_of_team[i], bat_pitch = 'Batter')
                    #print(df_advanced_player)
                    df_read = df_read.append(df_advanced_player)
            
                if stat_cat == 'batted-ball':
                    df_battedball_player = battedball_table(table_battedball, stat_cat, stat, df_battedball, team = team_name_formal, player = all_active_players_of_team[i], bat_pitch = 'Batter')
                    #print(df_battedball_player)
                    df_read = df_read.append(df_battedball_player)
        if (stat_cat == 'advanced') and (len(df_read) != orig_file_size):
            print(len(df_read) - orig_file_size)
            
            df_read.to_csv(team_to_check + '_' + str(season) + "_bat_advanced.csv" ,index = False)
            print('export done')
        if (stat_cat == 'batted-ball') and (len(df_read) != orig_file_size):    
            print(len(df_read) - orig_file_size)
            df_read.to_csv(team_to_check + '_' + str(season) + "_bat_battedball.csv",index = False)
            print('export done')
    driver.quit()

# Handedness Report

In [21]:
def get_player_url_handedness(roster_table):
    player = []
    player_url = []
    player_pos = []
    player_hand = []
    
    dcrt = roster_table.find_all('div',{"class":"depth-charts__roster-tables"})
    
    # first seven tables are the 40 man roster
    for i in range(7):        
        for row2 in dcrt[i].tbody.find_all('td',{"data-stat":"PLAYER"}):
                    
            #player list
            player_tmp = row2.text
            player.append(player_tmp)
                    
            #player url list
            player_url_tmp = row2.find('a')['href']
            size = len(player_url_tmp)
            player_url_tmp = 'https:' + player_url_tmp[:size - 5]
            player_url.append(player_url_tmp)
    for i in range(7): 
        for row3 in dcrt[i].tbody.find_all('td',{"data-stat":"POS"}):
            player_pos_tmp = row3.text
            player_pos.append(player_pos_tmp)
    for i in range(7): 
        for row4 in dcrt[i].tbody.find_all('tr'):
            if row4.find('td',{"data-stat":"THR"}) is not None:
                player_hand_tmp = row4.find('td',{"data-stat":"THR"}).text
            else:
                player_hand_tmp = row4.find('td',{"data-stat":"BATS"}).text
            player_hand.append(player_hand_tmp)
            
    rightLen = len(player)
    player_pos =  player_pos[:rightLen]
    player_hand = player_hand[:rightLen]

    return player, player_url, player_pos, player_hand

def get_all_players_handedness(teams):
    geckodriver = "C:\Program Files (x86)\geckodriver"
    os.environ["webdriver.gecko.driver"] = geckodriver
    
    driver = webdriver.Firefox(executable_path = geckodriver)

    # Get all players
    player_all = []
    player_url_all = []
    player_pos_all = []
    player_hand_all = []
    team_all = []
    team_abbr = []
    df_team_abbr = load_team_mapping()
    for team in teams:
        team_abbr_tmp = df_team_abbr[df_team_abbr['Teams'] == team]['Teams_abbr'].to_string()[5:]
        roster_url = get_roster_url(team)
        roster_table = get_roster_source(roster_url, driver)
    
        player, player_url, player_pos, player_hand = get_player_url_handedness(roster_table)
        for p in player:
            player_all.append(p)
            team_all.append(team)
            team_abbr.append(team_abbr_tmp)
        for u in player_url:
            player_url_all.append(u)
            
        for pos in player_pos:
            player_pos_all.append(pos)
        for hand in player_hand:
            player_hand_all.append(hand)
    
    print(len(player_all))
    print(player_all)
    print(len(player_url_all))
    print(len(team_all))
    print(len(team_abbr))
    print(len(player_pos_all))
    print(len(player_hand_all))    
    print(player_pos_all)
    print(player_hand_all)  
    df_player_all = pd.DataFrame({'Player':player_all, 'Player_url': player_url_all, 'Team':team_all, 'Team_abbr': team_abbr,
                                 'Player_pos':player_pos_all, 'Player_hand':player_hand_all})       
    return df_player_all

In [38]:
teams =  ['orioles','red sox','white sox', 'guardians', 'tigers', 'astros', 'royals', 'angels', 'twins', 'yankees',
            'athletics', 'mariners' , 'rays', 'rangers', 'blue jays', 'diamondbacks', 'braves','cubs','reds',
             'rockies','dodgers', 'marlins', 'brewers', 'mets', 'phillies', 'pirates', 'padres', 'giants', 'cardinals', 
             'nationals']
handPosition = get_all_players_handedness(teams)

1040
['Cedric Mullins', 'Trey Mancini', 'Anthony Santander', 'Ryan Mountcastle', 'Austin Hays', 'Adley Rutschman', 'Ramón Urías', 'Rougned Odor', 'Jorge Mateo', 'Robinson Chirinos', 'Jonathan Araúz', 'Tyler Nevin', 'Ryan McKenna', 'Jordan Lyles', 'Spenser Watkins', 'Tyler Wells', 'Dean Kremer', 'Austin Voth', 'Jorge López', 'Félix Bautista', 'Dillon Tate', 'Cionel Pérez', 'Joey Krehbiel', 'Keegan Akin', 'Bryan Baker', 'Rico Garcia', 'Kyle Bradish', 'Chris Ellis', 'John Means', 'Louis Head', 'Travis Lakins Sr.', 'Alexander Wells', 'Jarren Duran', 'Rafael Devers', 'J.D. Martinez', 'Xander Bogaerts', 'Alex Verdugo', 'Trevor Story', 'Franchy Cordero', 'Christian Vázquez', 'Jackie Bradley Jr.', 'Kevin Plawecki', 'Bobby Dalbec', 'Jeter Downs', 'Rob Refsnyder', 'Nick Pivetta', 'Chris Sale', 'Kutter Crawford', 'Tanner Houck', 'John Schreiber', 'Matt Strahm', 'Jake Diekman', 'Austin Davis', 'Hirokazu Sawamura', 'Ryan Brasier', 'Kaleb Ort', 'Darwinzon Hernandez', 'Phillips Valdez', 'Christian Ar

In [41]:
handPosition.to_csv('T:\Baseball\handednessReport.csv',index = False)

# Convert accented characters

In [32]:
def replace_accents(pitch_or_bat):
    directory = 'T:\Baseball\\export\\6_15_export\\2022_' + pitch_or_bat
    os.chdir(directory)
    for filename in os.listdir(directory):
        if 'csv' not in filename:
            continue
        team_to_check = filename[:filename.index('2022')]
        f = os.path.join(directory, filename)
        df_read = pd.read_csv(f)
        for i in range(len(df_read)):
            df_read['Player'][i] = unidecode.unidecode(df_read['Player'][i])
        df_read.to_csv(team_to_check + "2022 " + pitch_or_bat + ".csv" , index = False)
        print('done')

In [112]:
df_team_abbr = pd.read_csv("fangraphsExport\\teamMapping\\teams_mapping.csv")

In [22]:
teams =  ['orioles','red sox','white sox', 'guardians', 'tigers', 'astros', 'royals', 'angels', 'twins', 'yankees',
            'athletics', 'mariners' , 'rays', 'rangers', 'blue jays', 'diamondbacks', 'braves','cubs','reds',
             'rockies','dodgers', 'marlins', 'brewers', 'mets', 'phillies', 'pirates', 'padres', 'giants', 'cardinals', 
             'nationals']

df_player_all = get_all_players(teams)

In [20]:
# Find missing pitchersa advanced

directory = 'fangraphsExport\\2022_pitch_advanced'
stat = ['Handedness']
find_missing_pitchers(directory, df_player_all, 'advanced', stat, '2022')

NameError: name 'df_player_all' is not defined

In [ ]:
# Find missing pitcher battedball
directory = 'fangraphsExport\\2022_pitch_battedball'
stat = ['Handedness']
find_missing_pitchers(directory, df_player_all, 'batted-ball', stat, '2022')

In [ ]:
# Find missing batters advanced
directory = 'fangraphsExport\\2022_bat_advanced'
stat = ['Handedness']
find_missing_batters(directory, df_player_all, 'advanced', stat, '2022')

In [62]:
# Find missing batters battedball
directory = 'fangraphsExport\\2022_bat_battedball'
stat = ['Handedness']
find_missing_batters(directory, df_player_all, 'batted-ball', stat, '2022')

angels Reid Detmers
https://www.fangraphs.com/players/spenser-watkins/17489/ table_advanced is None
angels Patrick Sandoval
https://www.fangraphs.com/players/tyler-wells/20000/ table_advanced is None
angels José Suarez
https://www.fangraphs.com/players/dean-kremer/19350/ table_advanced is None
angels Noah Syndergaard
https://www.fangraphs.com/players/austin-voth/15047/ table_advanced is None
angels Raisel Iglesias
https://www.fangraphs.com/players/jorge-lopez/14527/ table_advanced is None
angels Ryan Tepera
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
angels Aaron Loup
https://www.fangraphs.com/players/dillon-tate/17796/ table_advanced is None
angels José Quijada
https://www.fangraphs.com/players/cionel-perez/19614/ table_advanced is None
angels Austin Warren
https://www.fangraphs.com/players/joey-krehbiel/13707/ table_advanced is None
angels Andrew Wantz
https://www.fangraphs.com/players/keegan-akin/19362/ table_advanced is None
angels Jose Marte
http

braves Kirby Yates
https://www.fangraphs.com/players/alexander-wells/19845/ table_advanced is None
brewers Corbin Burnes
https://www.fangraphs.com/players/jordan-lyles/7593/ table_advanced is None
brewers Brandon Woodruff
https://www.fangraphs.com/players/spenser-watkins/17489/ table_advanced is None
brewers Eric Lauer
https://www.fangraphs.com/players/tyler-wells/20000/ table_advanced is None
brewers Jason Alexander
https://www.fangraphs.com/players/dean-kremer/19350/ table_advanced is None
brewers Aaron Ashby
https://www.fangraphs.com/players/austin-voth/15047/ table_advanced is None
brewers Josh Hader
https://www.fangraphs.com/players/jorge-lopez/14527/ table_advanced is None
brewers Devin Williams
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
brewers Brad Boxberger
https://www.fangraphs.com/players/dillon-tate/17796/ table_advanced is None
brewers Trevor Gott
https://www.fangraphs.com/players/cionel-perez/19614/ table_advanced is None
brewers Brent 

dodgers Andrew Heaney
https://www.fangraphs.com/players/travis-lakins-sr/18600/ table_advanced is None
dodgers Dustin May
https://www.fangraphs.com/players/alexander-wells/19845/ table_advanced is None
dodgers Danny Duffy
dodgers Victor González
dodgers Brusdar Graterol
dodgers Daniel Hudson
dodgers Tommy Kahnle
dodgers Jimmy Nelson
dodgers Blake Treinen
28
export done
giants Logan Webb
https://www.fangraphs.com/players/spenser-watkins/17489/ table_advanced is None
giants Carlos Rodón
https://www.fangraphs.com/players/tyler-wells/20000/ table_advanced is None
giants Alex Wood
https://www.fangraphs.com/players/dean-kremer/19350/ table_advanced is None
giants Alex Cobb
https://www.fangraphs.com/players/austin-voth/15047/ table_advanced is None
giants Camilo Doval
https://www.fangraphs.com/players/jorge-lopez/14527/ table_advanced is None
giants Dominic Leone
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
giants Tyler Rogers
https://www.fangraphs.com/player

mets Joey Lucchesi
https://www.fangraphs.com/players/john-means/16269/ table_advanced is None
mets Tylor Megill
https://www.fangraphs.com/players/louis-head/13259/ table_advanced is None
mets John Curtiss
https://www.fangraphs.com/players/travis-lakins-sr/18600/ table_advanced is None
mets Trevor May
https://www.fangraphs.com/players/alexander-wells/19845/ table_advanced is None
mets Sean Reid-Foley
8
export done
nationals Josiah Gray
https://www.fangraphs.com/players/jordan-lyles/7593/ table_advanced is None
nationals Erick Fedde
https://www.fangraphs.com/players/spenser-watkins/17489/ table_advanced is None
nationals Aníbal Sánchez
https://www.fangraphs.com/players/tyler-wells/20000/ table_advanced is None
nationals Patrick Corbin
https://www.fangraphs.com/players/dean-kremer/19350/ table_advanced is None
nationals Paolo Espino
https://www.fangraphs.com/players/austin-voth/15047/ table_advanced is None
nationals Kyle Finnegan
https://www.fangraphs.com/players/jorge-lopez/14527/ table

pirates Yerry De Los Santos
https://www.fangraphs.com/players/jorge-lopez/14527/ table_advanced is None
pirates Wil Crowe
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
pirates Chris Stratton
https://www.fangraphs.com/players/dillon-tate/17796/ table_advanced is None
pirates Duane Underwood Jr.
https://www.fangraphs.com/players/cionel-perez/19614/ table_advanced is None
pirates Dillon Peters
https://www.fangraphs.com/players/joey-krehbiel/13707/ table_advanced is None
pirates Chase De Jong
https://www.fangraphs.com/players/keegan-akin/19362/ table_advanced is None
pirates Manny Bañuelos
https://www.fangraphs.com/players/bryan-baker/19804/ table_advanced is None
pirates Tyler Beede
https://www.fangraphs.com/players/rico-garcia/20023/ table_advanced is None
pirates Greg Allen
https://www.fangraphs.com/players/chris-ellis/16125/ table_advanced is None
pirates Max Kranick
https://www.fangraphs.com/players/travis-lakins-sr/18600/ table_advanced is None
pirate

rockies Alex Colomé
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
rockies Lucas Gilbreath
https://www.fangraphs.com/players/dillon-tate/17796/ table_advanced is None
rockies Robert Stephenson
https://www.fangraphs.com/players/cionel-perez/19614/ table_advanced is None
rockies Jake Bird
https://www.fangraphs.com/players/joey-krehbiel/13707/ table_advanced is None
rockies Jhoulys Chacín
https://www.fangraphs.com/players/keegan-akin/19362/ table_advanced is None
rockies Chad Smith
https://www.fangraphs.com/players/bryan-baker/19804/ table_advanced is None
rockies Ty Blach
https://www.fangraphs.com/players/rico-garcia/20023/ table_advanced is None
rockies Helcris Olivarez
https://www.fangraphs.com/players/chris-ellis/16125/ table_advanced is None
rockies Ryan Rolison
https://www.fangraphs.com/players/john-means/16269/ table_advanced is None
rockies Antonio Senzatela
https://www.fangraphs.com/players/louis-head/13259/ table_advanced is None
rockies Tyler Kin

yankees Jordan Montgomery
https://www.fangraphs.com/players/tyler-wells/20000/ table_advanced is None
yankees Jameson Taillon
https://www.fangraphs.com/players/dean-kremer/19350/ table_advanced is None
yankees Clay Holmes
https://www.fangraphs.com/players/austin-voth/15047/ table_advanced is None
yankees Michael King
https://www.fangraphs.com/players/jorge-lopez/14527/ table_advanced is None
yankees Wandy Peralta
https://www.fangraphs.com/players/felix-bautista/20666/ table_advanced is None
yankees Aroldis Chapman
https://www.fangraphs.com/players/dillon-tate/17796/ table_advanced is None
yankees Jonathan Loáisiga
https://www.fangraphs.com/players/cionel-perez/19614/ table_advanced is None
yankees Miguel Castro
https://www.fangraphs.com/players/joey-krehbiel/13707/ table_advanced is None
yankees Lucas Luetge
https://www.fangraphs.com/players/keegan-akin/19362/ table_advanced is None
yankees Albert Abreu
https://www.fangraphs.com/players/bryan-baker/19804/ table_advanced is None
yankees